# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff09b4fc640>
├── 'a' --> tensor([[ 0.7829,  0.6646, -1.7867],
│                   [-0.5245,  0.0034,  0.7927]])
└── 'x' --> <FastTreeValue 0x7ff09b4fc310>
    └── 'c' --> tensor([[ 0.1074, -0.9879, -1.6188, -1.0963],
                        [-0.2740, -0.0236,  0.7456,  0.7840],
                        [-0.6988,  0.1330, -1.0434, -0.0484]])

In [4]:
t.a

tensor([[ 0.7829,  0.6646, -1.7867],
        [-0.5245,  0.0034,  0.7927]])

In [5]:
%timeit t.a

85.7 ns ± 0.354 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff09b4fc640>
├── 'a' --> tensor([[-0.5010,  1.1006,  0.4434],
│                   [-0.1253, -0.0088, -1.8695]])
└── 'x' --> <FastTreeValue 0x7ff09b4fc310>
    └── 'c' --> tensor([[ 0.1074, -0.9879, -1.6188, -1.0963],
                        [-0.2740, -0.0236,  0.7456,  0.7840],
                        [-0.6988,  0.1330, -1.0434, -0.0484]])

In [7]:
%timeit t.a = new_value

85.4 ns ± 0.365 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7829,  0.6646, -1.7867],
               [-0.5245,  0.0034,  0.7927]]),
    x: Batch(
           c: tensor([[ 0.1074, -0.9879, -1.6188, -1.0963],
                      [-0.2740, -0.0236,  0.7456,  0.7840],
                      [-0.6988,  0.1330, -1.0434, -0.0484]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7829,  0.6646, -1.7867],
        [-0.5245,  0.0034,  0.7927]])

In [11]:
%timeit b.a

75.6 ns ± 0.412 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2014, -0.4798, -1.6550],
               [-0.5861,  0.6366, -1.3671]]),
    x: Batch(
           c: tensor([[ 0.1074, -0.9879, -1.6188, -1.0963],
                      [-0.2740, -0.0236,  0.7456,  0.7840],
                      [-0.6988,  0.1330, -1.0434, -0.0484]]),
       ),
)

In [13]:
%timeit b.a = new_value

644 ns ± 6.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.03 µs ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.2 µs ± 68.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

167 µs ± 823 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

167 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fefc0d1de50>
├── 'a' --> tensor([[[ 0.7829,  0.6646, -1.7867],
│                    [-0.5245,  0.0034,  0.7927]],
│           
│                   [[ 0.7829,  0.6646, -1.7867],
│                    [-0.5245,  0.0034,  0.7927]],
│           
│                   [[ 0.7829,  0.6646, -1.7867],
│                    [-0.5245,  0.0034,  0.7927]],
│           
│                   [[ 0.7829,  0.6646, -1.7867],
│                    [-0.5245,  0.0034,  0.7927]],
│           
│                   [[ 0.7829,  0.6646, -1.7867],
│                    [-0.5245,  0.0034,  0.7927]],
│           
│                   [[ 0.7829,  0.6646, -1.7867],
│                    [-0.5245,  0.0034,  0.7927]],
│           
│                   [[ 0.7829,  0.6646, -1.7867],
│                    [-0.5245,  0.0034,  0.7927]],
│           
│                   [[ 0.7829,  0.6646, -1.7867],
│                    [-0.5245,  0.0034,  0.7927]]])
└── 'x' --> <FastTreeValue 0x7fefc0d1de80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38 µs ± 238 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff098481460>
├── 'a' --> tensor([[ 0.7829,  0.6646, -1.7867],
│                   [-0.5245,  0.0034,  0.7927],
│                   [ 0.7829,  0.6646, -1.7867],
│                   [-0.5245,  0.0034,  0.7927],
│                   [ 0.7829,  0.6646, -1.7867],
│                   [-0.5245,  0.0034,  0.7927],
│                   [ 0.7829,  0.6646, -1.7867],
│                   [-0.5245,  0.0034,  0.7927],
│                   [ 0.7829,  0.6646, -1.7867],
│                   [-0.5245,  0.0034,  0.7927],
│                   [ 0.7829,  0.6646, -1.7867],
│                   [-0.5245,  0.0034,  0.7927],
│                   [ 0.7829,  0.6646, -1.7867],
│                   [-0.5245,  0.0034,  0.7927],
│                   [ 0.7829,  0.6646, -1.7867],
│                   [-0.5245,  0.0034,  0.7927]])
└── 'x' --> <FastTreeValue 0x7ff0984812e0>
    └── 'c' --> tensor([[ 0.1074, -0.9879, -1.6188, -1.0963],
                        [-0.2740, -0.0236,  0.7456,  0.7840],
                 

In [23]:
%timeit t_cat(trees)

35.4 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76.8 µs ± 207 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.7829,  0.6646, -1.7867],
                [-0.5245,  0.0034,  0.7927]],
       
               [[ 0.7829,  0.6646, -1.7867],
                [-0.5245,  0.0034,  0.7927]],
       
               [[ 0.7829,  0.6646, -1.7867],
                [-0.5245,  0.0034,  0.7927]],
       
               [[ 0.7829,  0.6646, -1.7867],
                [-0.5245,  0.0034,  0.7927]],
       
               [[ 0.7829,  0.6646, -1.7867],
                [-0.5245,  0.0034,  0.7927]],
       
               [[ 0.7829,  0.6646, -1.7867],
                [-0.5245,  0.0034,  0.7927]],
       
               [[ 0.7829,  0.6646, -1.7867],
                [-0.5245,  0.0034,  0.7927]],
       
               [[ 0.7829,  0.6646, -1.7867],
                [-0.5245,  0.0034,  0.7927]]]),
    x: Batch(
           c: tensor([[[ 0.1074, -0.9879, -1.6188, -1.0963],
                       [-0.2740, -0.0236,  0.7456,  0.7840],
                       [-0.6988,  0.1330, -1.0434, -0.0484]],
         

In [26]:
%timeit Batch.stack(batches)

96.9 µs ± 713 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.7829,  0.6646, -1.7867],
               [-0.5245,  0.0034,  0.7927],
               [ 0.7829,  0.6646, -1.7867],
               [-0.5245,  0.0034,  0.7927],
               [ 0.7829,  0.6646, -1.7867],
               [-0.5245,  0.0034,  0.7927],
               [ 0.7829,  0.6646, -1.7867],
               [-0.5245,  0.0034,  0.7927],
               [ 0.7829,  0.6646, -1.7867],
               [-0.5245,  0.0034,  0.7927],
               [ 0.7829,  0.6646, -1.7867],
               [-0.5245,  0.0034,  0.7927],
               [ 0.7829,  0.6646, -1.7867],
               [-0.5245,  0.0034,  0.7927],
               [ 0.7829,  0.6646, -1.7867],
               [-0.5245,  0.0034,  0.7927]]),
    x: Batch(
           c: tensor([[ 0.1074, -0.9879, -1.6188, -1.0963],
                      [-0.2740, -0.0236,  0.7456,  0.7840],
                      [-0.6988,  0.1330, -1.0434, -0.0484],
                      [ 0.1074, -0.9879, -1.6188, -1.0963],
                      [-0.2740, -

In [28]:
%timeit Batch.cat(batches)

176 µs ± 586 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

406 µs ± 9.79 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
